In [ ]:
import sys
sys.path.append("..")
import torch
from common_utils import DATA_HOME
from lstm.sales_data import Sales_Dataset

sd = Sales_Dataset(f"{DATA_HOME}\sales_data")
len(sd)

In [ ]:
# print(sd.O.describe())
# print(sd.TR.describe())
# print(sd.TS.describe())
# print(sd.S.describe())

In [ ]:
# print(sd.TR[sd.TR.index == (1, "2014-01-01")])
for i in range(len(sd)):
    test = sd[i]

print(test)

In [ ]:
%debug
